In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
import NUTILS as nutils
import matplotlib.pyplot as plt

In [2]:
# Prepare data sets
df_t = nutils.load_data('./train_full.csv')
df_s = nutils.load_data('./test_full.csv')

train_set, test_set = train_test_split(df_t, test_size=0.2, random_state=42)

X_full = nutils.encode(nutils.drop_features(df_t))
y_form_full = df_t["formation_energy_ev_natom"]
y_band_full = df_t["bandgap_energy_ev"]

X_train = nutils.encode(nutils.drop_features(train_set))
X_test  = nutils.encode(nutils.drop_features(test_set))
X_submt = nutils.encode(nutils.drop_features_submt(df_s))
                       
y_form = train_set["formation_energy_ev_natom"]
y_band = train_set["bandgap_energy_ev"]
                       
y_form_test = test_set["formation_energy_ev_natom"]
y_band_test = test_set["bandgap_energy_ev"]

y_comb = np.column_stack((y_form_full.values, y_band_full.values))
y_comb_train = np.column_stack((y_form.values, y_band.values))
y_comb_test = np.column_stack((y_form_test.values, y_band_test.values))

In [5]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, BatchNormalization
from keras.optimizers import SGD
import keras.losses as losses
from keras import regularizers

# For custom metrics
import keras.backend as K

def rmsle(y_actual, y_pred):
    return K.sqrt(losses.mean_squared_logarithmic_error(y_actual, y_pred))

Using TensorFlow backend.


In [ ]:
model_f = Sequential()
model_f.add(Dense(140, activation='elu', kernel_initializer='he_uniform', kernel_regularizer=None, input_dim=X_train.shape[1]))
model_f.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
model_f.add(Dropout(0.2))
model_f.add(Dense(140, activation='elu', kernel_initializer='he_uniform', kernel_regularizer=None))
model_f.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
model_f.add(Dropout(0.2))
model_f.add(Dense(140, activation='elu', kernel_initializer='he_uniform', kernel_regularizer=None))
model_f.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
model_f.add(Dropout(0.2))
model_f.add(Dense(140, activation='elu', kernel_initializer='he_uniform', kernel_regularizer=None))
model_f.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
model_f.add(Dropout(0.2))
model_f.add(Dense(140, activation='elu', kernel_initializer='he_uniform', kernel_regularizer=None))
model_f.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
model_f.add(Dropout(0.2))
model_f.add(Dense(140, activation='elu', kernel_initializer='he_uniform', kernel_regularizer=None))
model_f.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
model_f.add(Dropout(0.2))
model_f.add(Dense(140, activation='elu', kernel_initializer='he_uniform', kernel_regularizer=None))
model_f.add(BatchNormalization(axis=-1, momentum=0.99, epsilon=0.001, center=True, scale=True))
model_f.add(Dropout(0.2))
model_f.add(Dense(2))
#sgd = SGD(lr=0.0001, decay=1e-7, momentum=0.9, nesterov=True)

sgd = SGD(lr=0.001, momentum=0.9)
model_f.compile(optimizer=keras.optimizers.Adam(lr=0.001),
              loss=losses.mean_absolute_error,
              metrics=[rmsle])

history_f = model_f.fit(X_full.values, y_comb,
              epochs=2200,
              validation_split=0.2,
              batch_size=X_full.shape[0])
              #callbacks=keras_callbacks)

In [ ]:
#print(score_f)
plt.plot(history_f.history['loss'])
plt.plot(history_f.history['rmsle'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
submit_pred_form = model_f.predict(X_submt.values)
submit_df = pd.DataFrame(submit_pred_form, columns=['formation_energy_ev_natom', "bandgap_energy_ev"])
submit_df[submit_df < 0] = 0
submit_df.insert(0, 'id', range(1, 601))

# Save to file
submit_df.to_csv("./k_dnn_7.csv", index=False)